In [14]:
import numpy as np
import pandas as pd
from pandas import DataFrame, pivot_table, read_csv

from rxtrade.data import add_amplitude_column, add_basic_columns
from rxtrade.utils import get_data_files

In [15]:
def add_stats_columns(ticker: str, file_path: str) -> DataFrame:
    df = read_csv(file_path)

    add_basic_columns(df)
    add_amplitude_column(ticker, df)

    df["Market Open"] = np.where(
        ((df["time"].dt.hour == 8) & (df["time"].dt.minute >= 30)) |
        ((df["time"].dt.hour == 9) & (df["time"].dt.minute < 15)),
        "O",
        "N"
    )

    df[f"Close ({ticker})"] = df["close"]
    df[f"High ({ticker})"] = df["high"]
    df[f"Low ({ticker})"] = df["low"]
    return df

In [16]:
def process_for_pivot(ticker: str, df: DataFrame) -> None:
    df[f"Amplitude ({ticker})"] = df[f"Amplitude ({ticker})"] / (df[f"Close ({ticker})"] / df[f"Close (NQ)"])

In [17]:
def make_pivot(tickers: list[str], df: DataFrame) -> DataFrame:
    return pivot_table(
        df,
        values=[f"High ({ticker})" for ticker in tickers] + [f"Low ({ticker})" for ticker in tickers],
        index=["Market Date", "Market Code", "Market Open"],
        aggfunc={
            f"High ({ticker})": np.max for ticker in tickers
        } | {
            f"Low ({ticker})": np.min for ticker in tickers
        },
        dropna=False,
    )

In [47]:
from IPython.core.display_functions import display

tickers = []
df = DataFrame(columns=["time"])

pd.options.display.max_rows = 999

for data_file in get_data_files("./archive/futures/**/*.csv"):
    if data_file.period != 5:
        continue

    tickers.append(data_file.ticker)
    ticker_df = add_stats_columns(data_file.ticker, data_file.file_path)

    if len(df):
        df = df.merge(ticker_df, left_on="time", right_on="time")
    else:
        df = ticker_df

for ticker in tickers:
    process_for_pivot(ticker, df)

pivot = make_pivot(tickers, df)

for ticker in tickers:
    print(ticker)
    print((pivot.loc[(slice(None), "R", "O")][f"High ({ticker})"] < pivot.loc[(slice(None), "E", "N")][f"High ({ticker})"]).value_counts())
    print((pivot.loc[(slice(None), "R", "O")][f"Low ({ticker})"] > pivot.loc[(slice(None), "E", "N")][f"Low ({ticker})"]).value_counts())
    print()

ES
True     27
False    12
Name: High (ES), dtype: int64
True     28
False    11
Name: Low (ES), dtype: int64

NQ
True     28
False    11
Name: High (NQ), dtype: int64
True     27
False    12
Name: Low (NQ), dtype: int64

YM
True     28
False    11
Name: High (YM), dtype: int64
True     26
False    13
Name: Low (YM), dtype: int64



In [19]:
pivot[pivot.index.get_level_values("Market Code") == "R"]["Amplitude (NQ)"].plot.line()


KeyError: 'Amplitude (NQ)'